In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
window_size = 5

### 준비

In [3]:
transaction_all = pd.read_csv('../../데이터/transaction/transaction_all.csv')

transaction_all['계약연도'] = transaction_all['계약년월'].apply(lambda x: str(x)[:4])
transaction_all['거래금액(만원)'] = transaction_all['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
transaction_all['제곱미터당 거래금액(만원)'] = transaction_all['거래금액(만원)'] / transaction_all['전용면적(㎡)']
transaction_all['단지'] = transaction_all['시군구'] + ' ' + transaction_all['단지명']
transaction_all = transaction_all[['단지', '계약년월', '건축년도', '제곱미터당 거래금액(만원)']]

In [4]:
economy_all = pd.read_excel('../../데이터/Economy/economy_all.xlsx')
date_range = pd.date_range(start="2006-01", periods=len(economy_all), freq="M")
economy_all['계약년월'] = date_range.strftime('%Y%m').astype(int)

In [5]:
print(len(transaction_all))
print(len(transaction_all['단지'].unique()))

1237568
8977


### 동일 동,단지이지만 건축년도가 다른 단지 제거

In [6]:
transaction_all = transaction_all.groupby('단지').filter(lambda x: x['건축년도'].nunique() == 1)

### 거개량 적은(window_size+1 미만) 단지 제거

In [7]:
apartment_complex_volume = transaction_all.groupby('단지').agg({'건축년도':'first','제곱미터당 거래금액(만원)':'count'}).reset_index().rename(columns={'제곱미터당 거래금액(만원)':'거래량'})
small_volume_apartment_complexes = apartment_complex_volume[apartment_complex_volume['거래량'] < window_size+1]['단지'].to_list()

transaction_all = transaction_all[~transaction_all['단지'].isin(small_volume_apartment_complexes)].reset_index(drop=True)

In [8]:
print(len(transaction_all))
print(len(transaction_all['단지'].unique()))

1233257
8326


### 이상치 제거

In [9]:
transaction_all['Z 스코어'] = transaction_all.groupby('단지')['제곱미터당 거래금액(만원)'].transform(lambda x: (x-x.mean()) / x.std())
transaction_all = transaction_all[transaction_all['Z 스코어'].between(-3, 3)]
transaction_all = transaction_all.drop(columns=['Z 스코어']).reset_index(drop=True)

In [10]:
print(len(transaction_all))
print(len(transaction_all['단지'].unique()))

1208749
8323


### 동일 단지,계약년월일 경우, 제곱미터당 거래금액(만원) 평균

In [11]:
transaction_all = transaction_all.groupby(['단지','계약년월','건축년도'])['제곱미터당 거래금액(만원)'].mean().to_frame().reset_index()

In [12]:
print(len(transaction_all))
print(len(transaction_all['단지'].unique()))

417224
8323


### 동과 단지 분리 후 저장

In [13]:
transaction_all['동'] = transaction_all['단지'].apply(lambda x: x.split(' ')[:3]).apply(lambda x: ' '.join(x))
transaction_all['단지'] = transaction_all['단지'].apply(lambda x: x.split(' ')[3:]).apply(lambda x: ' '.join(x))
transaction_all = transaction_all[['동','단지','건축년도','계약년월','제곱미터당 거래금액(만원)']]

In [14]:
transaction_all.to_excel('transaction_all.xlsx')
transaction_all.to_csv('transaction_all.csv')